It begins

In [107]:
# Imports

import numpy as np # linear algebra
import pandas as pd

# for string manipulation
import re

#for text processing
import re, string
#import nltk
#nltk.download('stopwords')
#from nltk.tokenize import word_tokenize
#from nltk.stem import SnowballStemmer
#from nltk.corpus import wordnet
#nltk.download('wordnet')
#nltk.download('punkt')
#nltk.download('averaged_perceptron_tagger')


# For visualizations
#import plotly

# Datasets

In [108]:
path = 'C:/Users/mucha/PycharmProjects/Score-Clinical-Patient-Notes/nbme-score-clinical-patient-notes/'
features = pd.read_csv(path + '/features.csv')
patient_notes = pd.read_csv(path + '/patient_notes.csv')
train = pd.read_csv(path + '/train.csv')
train = train.merge(features[['feature_num', 'feature_text']], left_on='feature_num', right_on='feature_num').sort_values(by=['case_num', 'pn_num'])
cols = train.columns.tolist()
reordered_cols = cols[:3] + cols[4:6] + cols[3:4] + cols[6:7]
train = train[reordered_cols]
test = pd.read_csv(path + '/test.csv')

In [109]:
patient_notes.head()

,pn_num,case_num,pn_history
0,0,0,"17-year-old male, has come to the student heal..."
1,1,0,17 yo male with recurrent palpitations for the...
2,2,0,Dillon Cleveland is a 17 y.o. male patient wit...
3,3,0,a 17 yo m c/o palpitation started 3 mos ago; \...
4,4,0,17yo male with no pmh here for evaluation of p...


In [110]:
train.head()

,id,case_num,pn_num,annotation,location,feature_num,feature_text
0,00016_000,0,16,['dad with recent heart attcak'],['696 724'],0,Family-history-of-MI-OR-Family-history-of-myoc...
100,00016_001,0,16,"['mom with ""thyroid disease']",['668 693'],1,Family-history-of-thyroid-disorder
200,00016_002,0,16,['chest pressure'],['203 217'],2,Chest-pressure
300,00016_003,0,16,"['intermittent episodes', 'episode']","['70 91', '176 183']",3,Intermittent-symptoms
400,00016_004,0,16,['felt as if he were going to pass out'],['222 258'],4,Lightheaded


# Data Cleaning
For some of our methods, we want cleaned, normalized text.

# Data Exploration

In [111]:
case_count = patient_notes['case_num'].nunique()
note_count = patient_notes.shape[0]
feature_count = features['feature_num'].nunique()
tagged_notes = train['pn_num'].nunique()

print("There are {cases:n} cases with {notes:n} notes".format(cases = case_count, notes = note_count))
print("{tagged:n} notes are represented in the training set".format(tagged = tagged_notes))
print("There are {features:n} features".format(features = feature_count))

There are 10 cases with 42146 notes
1000 notes are represented in the training set
There are 143 features


To get a sense of how these features are distributed, we can look at some basic statistical measurements of the data.

In [112]:
feature_dist = train[train['location'] != '[]']['feature_num'].value_counts()
feature_dist.describe()

count    143.000000
mean      69.237762
std       26.783419
min        1.000000
25%       53.000000
50%       76.000000
75%       91.000000
max      100.000000
Name: feature_num, dtype: float64

There is a significant quantity of features appearing tagged in at least 91 notes, a significant quantity appearing in less than 53, and a feature that only appears in diagnostic notes a single time.
From this, we can see that there's a stark imbalance between how many representative spans of text we have for some diagnostic features. While pre-trained neural networks may help find underrepresented classes regardless, we will consider methods to handle this imbalance, moving forward.

In [113]:
train[train['location'] != '[]']['case_num'].value_counts().sort_index()

0     998
1     942
2     998
3    1198
4     746
5    1174
6     971
7     692
8    1190
9     992
Name: case_num, dtype: int64

We have data for all 10 cases. There is also an imbalance here, though not as extreme. Since ideally we want a tool that accurately tags text spans without inference from contextual patient information, this imbalance shouldn't matter.

# Exact Match
As explored above, some classes are really, really, really, really underrepresented and stuff. I'mma get some more training data real quick, sit tight.


In [114]:
train

,id,case_num,pn_num,annotation,location,feature_num,feature_text
0,00016_000,0,16,['dad with recent heart attcak'],['696 724'],0,Family-history-of-MI-OR-Family-history-of-myoc...
100,00016_001,0,16,"['mom with ""thyroid disease']",['668 693'],1,Family-history-of-thyroid-disorder
200,00016_002,0,16,['chest pressure'],['203 217'],2,Chest-pressure
300,00016_003,0,16,"['intermittent episodes', 'episode']","['70 91', '176 183']",3,Intermittent-symptoms
400,00016_004,0,16,['felt as if he were going to pass out'],['222 258'],4,Lightheaded
...,...,...,...,...,...,...,...
13899,95333_912,9,95333,[],[],912,Family-history-of-migraines
13999,95333_913,9,95333,[],[],913,Female
14099,95333_914,9,95333,['photobia'],['274 282'],914,Photophobia
14199,95333_915,9,95333,['no sick contacts'],['421 437'],915,No-known-illness-contacts


In [154]:
features

,feature_num,case_num,feature_text
0,0,0,Family-history-of-MI-OR-Family-history-of-myoc...
1,1,0,Family-history-of-thyroid-disorder
2,2,0,Chest-pressure
3,3,0,Intermittent-symptoms
4,4,0,Lightheaded
...,...,...,...
138,912,9,Family-history-of-migraines
139,913,9,Female
140,914,9,Photophobia
141,915,9,No-known-illness-contacts


In [173]:
feature_identifiers = {} # Keys are feature numbers, values are sets full of every text annotation labeled as that feature number
for feature_id in features['feature_num'].unique():
    identifying_text = set()
    feature_annotations = train[train['feature_num'] == feature_id]['annotation'].unique().tolist()
    for note_annotation in feature_annotations:
        matchable_texts = eval(note_annotation)
        for text in matchable_texts:
            identifying_text.add(text)
    feature_identifiers[feature_id] = identifying_text
feature_identifiers

{0: {'CAD in father',
  'Dad MI',
  'Dad had MI',
  'Dad had heart attack',
  'Dad hx MI',
  'Dad with "heart attack',
  'Dad-MI',
  'Dad: Heart problem',
  'FATHER HAD AN MI',
  'FATHER HAD HEART ATTACK',
  'FATHER HAD MI',
  'FATHER HAS HEART PROBLEM',
  'FATHER MI',
  'FH positive for a recent heart attack',
  'Father - MI',
  'Father MI',
  'Father had MI',
  'Father had an MI',
  'Father had possible MI',
  'Father heart problem',
  'Father suffer an MI',
  'Father with MI',
  'Father- MI',
  'Father-recently had an MI',
  'Father: Myocardial infarction',
  'Heart attack 1 year ago - father',
  'Heart attack in father',
  'MI at 52 for father',
  'MI father',
  'MI in his father',
  'MI in the father',
  'MI of his father',
  'ather heart attack',
  'dad - heart attack',
  'dad MI',
  'dad had MI',
  'dad had a heart attack',
  'dad had heart attack',
  'dad had recent MI',
  'dad has cardiac issues',
  'dad with recent heart attack',
  'dad with recent heart attcak',
  'dad, hear

Any patient notes that already appear

In [178]:
already_annotated = train['pn_num'].unique().tolist()
trimmed_notes = patient_notes[~patient_notes['pn_num'].isin(already_annotated)]

,pn_num,case_num,pn_history
0,0,0,"17-year-old male, has come to the student heal..."
1,1,0,17 yo male with recurrent palpitations for the...
2,2,0,Dillon Cleveland is a 17 y.o. male patient wit...
3,3,0,a 17 yo m c/o palpitation started 3 mos ago; \...
4,4,0,17yo male with no pmh here for evaluation of p...
...,...,...,...
42139,95328,9,20 YO F C/O DULL HEADACHE SINCE YESTERDAY. SHE...
42140,95329,9,20 y/o F c/o headache that started yesterday m...
42142,95331,9,A 20 YO F CAME COMPLAIN A DULL 8/10 HEADACHE T...
42143,95332,9,Ms. Madden is a 20yo female who presents with ...


In [188]:
buh = []
for feature_num in feature_identifiers.keys():
        print(feature_identifiers[feature_num])
# Loops through all patient notes
for idx, row in trimmed_notes.iterrows():
    for feature_num in feature_identifiers.keys():
        if row['pn_history'].contains()
    print(row['pn_history'])

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [186]:
patient_notes

,pn_num,case_num,pn_history
0,0,0,"17-year-old male, has come to the student heal..."
1,1,0,17 yo male with recurrent palpitations for the...
2,2,0,Dillon Cleveland is a 17 y.o. male patient wit...
3,3,0,a 17 yo m c/o palpitation started 3 mos ago; \...
4,4,0,17yo male with no pmh here for evaluation of p...
...,...,...,...
42141,95330,9,Ms. Madden is a 20 yo female presenting w/ the...
42142,95331,9,A 20 YO F CAME COMPLAIN A DULL 8/10 HEADACHE T...
42143,95332,9,Ms. Madden is a 20yo female who presents with ...
42144,95333,9,Stephanie madden is a 20 year old woman compla...


In [162]:
train

,id,case_num,pn_num,annotation,location,feature_num,feature_text
0,00016_000,0,16,['dad with recent heart attcak'],['696 724'],0,Family-history-of-MI-OR-Family-history-of-myoc...
100,00016_001,0,16,"['mom with ""thyroid disease']",['668 693'],1,Family-history-of-thyroid-disorder
200,00016_002,0,16,['chest pressure'],['203 217'],2,Chest-pressure
300,00016_003,0,16,"['intermittent episodes', 'episode']","['70 91', '176 183']",3,Intermittent-symptoms
400,00016_004,0,16,['felt as if he were going to pass out'],['222 258'],4,Lightheaded
...,...,...,...,...,...,...,...
13899,95333_912,9,95333,[],[],912,Family-history-of-migraines
13999,95333_913,9,95333,[],[],913,Female
14099,95333_914,9,95333,['photobia'],['274 282'],914,Photophobia
14199,95333_915,9,95333,['no sick contacts'],['421 437'],915,No-known-illness-contacts


In [ ]:
# THIS IS GONNA BE A GOOD CELL, DON'T DELETE IT, TRUST ME
new_annotations = [] #




In [153]:
feature_annotations = train[train['feature_num'] == 0]['annotation'].unique().tolist()
feature_annotations

["['dad with recent heart attcak']",
 '[]',
 "['father: heart attack']",
 "['Father MI']",
 "['Dad-MI']",
 "['father had acute MI']",
 "['father heart attach']",
 "['dad had recent MI']",
 "['dad had heart attack']",
 "['Father heart problem']",
 "['MI of his father', 'Father had MI']",
 "['MI in the father']",
 "['father with heart attack recently', 'father with heart attack']",
 "['dad MI']",
 "['FATHER HAD AN MI']",
 "['dad had MI']",
 "['father hx of MI']",
 "['father had a heart attack']",
 "['Father had MI']",
 "['father with MI']",
 "['father MI']",
 "['father had heart attack']",
 "['Dad had heart attack']",
 "['Heart attack 1 year ago - father']",
 "['ather heart attack']",
 "['FATHER HAD MI']",
 "['father heart attack']",
 "['father had an MI']",
 "['Heart attack in father']",
 "['FATHER HAD HEART ATTACK']",
 "['MI at 52 for father']",
 "['dad - heart attack']",
 "['Father had an MI']",
 "['Father-recently had an MI']",
 "['father had MI']",
 "['FATHER MI']",
 "['Dad MI']",
 

In [138]:
for thing in train[train['feature_num'] == 6]['annotation'].unique().tolist():
    print(thing)
    print(type(thing))
    print(str_to_list(thing)[0])
    print(type(str_to_list(thing)))

['adderall', 'adderrall', 'adderrall']
<class 'str'>
adderall
<class 'list'>
[]
<class 'str'>

<class 'list'>
['adderall', 'Adderall']
<class 'str'>
adderall
<class 'list'>
['aderall']
<class 'str'>
aderall
<class 'list'>
['adderall']
<class 'str'>
adderall
<class 'list'>
['aderol', 'aderol', 'aderol']
<class 'str'>
aderol
<class 'list'>
['Adderall']
<class 'str'>
Adderall
<class 'list'>
['adderral']
<class 'str'>
adderral
<class 'list'>
['uses Adderral']
<class 'str'>
uses Adderral
<class 'list'>
['using Adderall']
<class 'str'>
using Adderall
<class 'list'>
['aderol']
<class 'str'>
aderol
<class 'list'>
['USES ADERALL', 'ADERALL']
<class 'str'>
USES ADERALL
<class 'list'>
['Addrell', 'addrell']
<class 'str'>
Addrell
<class 'list'>
['Adderol']
<class 'str'>
Adderol
<class 'list'>
['adderall', 'adderal', 'adderal', 'adderal']
<class 'str'>
adderall
<class 'list'>
['adderal']
<class 'str'>
adderal
<class 'list'>
['adderall', 'adderall']
<class 'str'>
adderall
<class 'list'>
['taking Add

In [134]:
str_to_list(train[train['feature_num'] == 0]['annotation'].tolist()[0])

['dad with recent heart attcak']

In [135]:
train[(train['feature_num']==0)&(train['case_num']==0)]['annotation'].unique()

array(["['dad with recent heart attcak']", '[]',
       "['father: heart attack']", "['Father MI']", "['Dad-MI']",
       "['father had acute MI']", "['father heart attach']",
       "['dad had recent MI']", "['dad had heart attack']",
       "['Father heart problem']",
       "['MI of his father', 'Father had MI']", "['MI in the father']",
       "['father with heart attack recently', 'father with heart attack']",
       "['dad MI']", "['FATHER HAD AN MI']", "['dad had MI']",
       "['father hx of MI']", "['father had a heart attack']",
       "['Father had MI']", "['father with MI']", "['father MI']",
       "['father had heart attack']", "['Dad had heart attack']",
       "['Heart attack 1 year ago - father']", "['ather heart attack']",
       "['FATHER HAD MI']", "['father heart attack']",
       "['father had an MI']", "['Heart attack in father']",
       "['FATHER HAD HEART ATTACK']", "['MI at 52 for father']",
       "['dad - heart attack']", "['Father had an MI']",
       "['F

In [122]:
known_annotations = {}
for feature_num in feature_nums:


0
1
2
3
4
5
6
7
8
9
10
11
12
100
101
102
103
104
105
106
107
108
109
110
111
112
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
400
401
402
403
404
405
406
407
408
409
500
501
502
503
504
505
506
507
508
509
510
511
512
513
514
515
516
517
600
601
602
603
604
605
606
607
608
609
610
611
700
701
702
703
704
705
706
707
708
800
801
802
803
804
805
806
807
808
809
810
811
812
813
814
815
816
817
900
901
902
903
904
905
906
907
908
909
910
911
912
913
914
915
916


In [118]:
test

,id,case_num,pn_num,feature_num
0,00016_000,0,16,0
1,00016_001,0,16,1
2,00016_002,0,16,2
3,00016_003,0,16,3
4,00016_004,0,16,4


In [119]:
train[train['feature_num'] == 0]['annotation'].tolist()

#annotations = list(filter([].__ne__, train[train['feature_num'] == 0]['annotation'].tolist()

#annotations = list(filter(('[]').__ne__, train[train['feature_num'] == 0]['annotation'].tolist()))

#annotations

["['dad with recent heart attcak']",
 '[]',
 "['father: heart attack']",
 "['Father MI']",
 "['Dad-MI']",
 "['father had acute MI']",
 "['father heart attach']",
 "['Father MI']",
 "['dad had recent MI']",
 "['dad had heart attack']",
 "['Father heart problem']",
 "['MI of his father', 'Father had MI']",
 "['MI in the father']",
 "['father with heart attack recently', 'father with heart attack']",
 "['dad MI']",
 "['FATHER HAD AN MI']",
 "['dad had MI']",
 "['father hx of MI']",
 "['father had a heart attack']",
 '[]',
 "['Father had MI']",
 "['father with MI']",
 "['father MI']",
 "['father had heart attack']",
 "['Dad had heart attack']",
 "['Heart attack 1 year ago - father']",
 "['ather heart attack']",
 "['FATHER HAD MI']",
 "['father heart attack']",
 "['father had an MI']",
 "['Heart attack in father']",
 "['FATHER HAD HEART ATTACK']",
 "['MI at 52 for father']",
 "['father MI']",
 "['dad - heart attack']",
 "['Father had an MI']",
 "['dad MI']",
 "['Father-recently had an MI']"

In [120]:
train

,id,case_num,pn_num,annotation,location,feature_num,feature_text
0,00016_000,0,16,['dad with recent heart attcak'],['696 724'],0,Family-history-of-MI-OR-Family-history-of-myoc...
100,00016_001,0,16,"['mom with ""thyroid disease']",['668 693'],1,Family-history-of-thyroid-disorder
200,00016_002,0,16,['chest pressure'],['203 217'],2,Chest-pressure
300,00016_003,0,16,"['intermittent episodes', 'episode']","['70 91', '176 183']",3,Intermittent-symptoms
400,00016_004,0,16,['felt as if he were going to pass out'],['222 258'],4,Lightheaded
...,...,...,...,...,...,...,...
13899,95333_912,9,95333,[],[],912,Family-history-of-migraines
13999,95333_913,9,95333,[],[],913,Female
14099,95333_914,9,95333,['photobia'],['274 282'],914,Photophobia
14199,95333_915,9,95333,['no sick contacts'],['421 437'],915,No-known-illness-contacts


In [121]:
from transformers import pipeline

#classifier = pipeline("sentiment-analysis")
classifier = pipeline("ner", grouped_entities = True)#, model = 'bert-base-NER')
classifier(
    [
        "beep Boop boop Beep Blorp canadian"
    ]
)

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english)
Some layers from the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing TFBertForTokenClassification: ['dropout_147']
- This IS expected if you are initializing TFBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForTokenClassification were initialized from the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english.
If your task is similar to the task the

[[{'entity_group': 'ORG',
   'score': 0.41625246,
   'word': '##adia',
   'start': 29,
   'end': 33}]]